In [85]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import math
from spotipy.exceptions import SpotifyException

# show all columns
pd.set_option('display.max_columns', None)

In [86]:
# Set up of Spotify API credentials
client_id = "15eeb8c96d3c4e38991a827042884398" #Audrey
client_secret = "a202e49dc9274a469c7d222428966c1d" #Audrey

# Setting input and output file names
input_file = "billboard_hot_100_2020_2023.csv"
output_file = "billboard_hot_100_audio_features.csv"

# Setting up the Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Reading the songs from the input CSV
songs_df = pd.read_csv("Spotify_TT.csv")

In [87]:
# Removing duplicates based on artist name
songs_df = songs_df.drop_duplicates(subset=['artist'], keep='first')

In [88]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11904 entries, 0 to 41545
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          11904 non-null  int64  
 1   track               11904 non-null  object 
 2   artist              11904 non-null  object 
 3   track_id            11904 non-null  object 
 4   danceability_x      11904 non-null  float64
 5   energy_x            11904 non-null  float64
 6   key_x               11904 non-null  object 
 7   loudness_x          11904 non-null  float64
 8   mode_x              11904 non-null  object 
 9   speechiness_x       11904 non-null  float64
 10  acousticness_x      11904 non-null  float64
 11  instrumentalness_x  11904 non-null  float64
 12  liveness_x          11904 non-null  float64
 13  valence_x           11904 non-null  float64
 14  tempo_x             11904 non-null  float64
 15  duration_ms         11904 non-null  int64  
 16  time

In [89]:
def get_genre_from_uri(uri):
    print("Getting track information...")
    try:
        track = sp.track(uri)
    except SpotifyException as e:
        if e.http_status == 429:
            retry_after = int(e.headers.get('Retry-After', 1))
            print(f"Rate limit exceeded. Waiting for {retry_after} seconds.")
            time.sleep(retry_after)
            return get_genre_from_uri(uri)
        else:
            print(f"Error while getting track information: {e}")
            return ""
    
    artist_id = track["artists"][0]["id"]
    
    print("Getting artist information...")
    artist = sp.artist(artist_id)
    print("Artist information received.")
    
    genres = artist["genres"]
    
    if genres:
        genre_str = ", ".join(genres)
        print(f"Added genres for track URI {uri}: {genre_str}")
        return genre_str
    else:
        print(f"No genres found for track URI {uri}")
        return ""

# Add genres as a new column to your dataframe
def add_genres_with_rate_limit(df, batch_size=100):
    num_batches = math.ceil(len(df) / batch_size)

    def process_batch(batch_df):
        for index, row in batch_df.iterrows():
            uri = row['track_id']
            batch_df.at[index, 'Genre'] = get_genre_from_uri(uri)
            time.sleep(1 / 3)  # 333 ms
        return batch_df

    for i in range(num_batches):
        print(f"Processing batch {i + 1} out of {num_batches}")
        start_index = i * batch_size
        end_index = start_index + batch_size
        df.iloc[start_index:end_index] = process_batch(df.iloc[start_index:end_index])
        
        # Save the processed batch to a CSV file
        df.iloc[start_index:end_index].to_csv(f"batch_{i + 1}.csv", index=False)

add_genres_with_rate_limit(songs_df)

Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 1dtKN6wwlolkM8XZy2y9C1: chicago soul, classic soul, southern soul
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 5hjsmSnUefdUqzsDogisiX: chanson, french rock
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 6uk8tI6pwxxdVTNlNOJeJh: calypso
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 7aNjMJ05FvUXACPWZ7yJmv: cha-cha-cha, cuban rumba, mambo, musica tradicional cubana, salsa
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 1rQ0clvgkzWr001POOPJWx: filmi, vintage tollywood
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 32VBSoD2vcoIOiPEvAfFXU: clas

KeyboardInterrupt: 

In [ ]:
# Add genres as a new column to your dataframe
songs_df['Genre'] = songs_df['track_id'].apply(get_genre_from_uri)

Getting track information...


KeyboardInterrupt: 

In [ ]:
songs_df.head(20)

,Unnamed: 0,track,artist,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target,TikTok_popular
0,0,Jealous Kind Of Fella,Garland Green,1dtKN6wwlolkM8XZy2y9C1,0.417,0.6200,3,-7.727,1,0.0403,0.4900,0.000000,0.0779,0.8450,185.655,173533,3,32.94975,9,1,NaN
1,1,Initials B.B.,Serge Gainsbourg,5hjsmSnUefdUqzsDogisiX,0.498,0.5050,3,-12.475,1,0.0337,0.0180,0.107000,0.1760,0.7970,101.801,213613,4,48.82510,10,0,NaN
2,2,Melody Twist,Lord Melody,6uk8tI6pwxxdVTNlNOJeJh,0.657,0.6490,5,-13.392,1,0.0380,0.8460,0.000004,0.1190,0.9080,115.940,223960,4,37.22663,12,0,NaN
3,3,Mi Bomba Sonó,Celia Cruz,7aNjMJ05FvUXACPWZ7yJmv,0.590,0.5450,7,-12.058,0,0.1040,0.7060,0.024600,0.0610,0.9670,105.592,157907,4,24.75484,8,0,NaN
4,4,Uravu Solla,P. Susheela,1rQ0clvgkzWr001POOPJWx,0.515,0.7650,11,-3.515,0,0.1240,0.8570,0.000872,0.2130,0.9060,114.617,245600,4,21.79874,14,0,NaN
5,5,Beat n. 3,Ennio Morricone,32VBSoD2vcoIOiPEvAfFXU,0.697,0.6730,0,-10.573,1,0.0266,0.7140,0.919000,0.1220,0.7780,112.117,167667,4,65.48604,7,0,NaN
6,6,Samba De Uma Nota So (One Note Samba),Antônio Carlos Jobim,62eoocmGk59EMfuRcLBvUL,0.662,0.2720,0,-18.883,1,0.0313,0.3600,0.228000,0.0963,0.5910,143.507,134360,4,47.82155,7,0,NaN
7,7,Happy Days,Marv Johnson,0ecGwZA1ReAq4Mn3ZInaT4,0.720,0.6240,5,-9.086,0,0.0473,0.7950,0.000000,0.4880,0.8870,119.999,160040,4,30.42891,8,1,NaN
8,8,Carolina - Remastered 2006,Caetano Veloso,15gNqKkZN9Mq9AvnRqWJKb,0.545,0.2200,2,-15.079,0,0.0828,0.5820,0.239000,0.2690,0.3860,118.223,158413,4,47.08099,6,0,NaN
9,9,I Can Hear Music,The Beach Boys,1hcMfYTsRTC4hIKbfosxjz,0.511,0.6030,2,-7.637,1,0.0280,0.0385,0.000002,0.1420,0.6850,128.336,157293,4,43.36534,9,1,NaN
